# SelectKBest
***

- 예측하고자 하는 target 변수와 다른 feature들 간의 상관관계를 고려하여 가장 강한 상관관계를 가지는 변수 k개를 선정
- 상관관계를 분석하는 방법으로는 f-regression, chi2, f-classif 등이 있다.

In [46]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

from sklearn.datasets import load_breast_cancer, load_boston
from sklearn.feature_selection import SelectKBest, chi2, f_regression, SelectFromModel
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

#### 1) Regression

In [35]:
data = load_boston()

In [36]:
X = pd.DataFrame(data.data, columns = data.feature_names)
y = data.target

In [37]:
X.shape

(506, 13)

In [38]:
rf = RandomForestRegressor(random_state = 42)

In [39]:
sk = SelectKBest(f_regression, k = 10)
sk_X = sk.fit_transform(X, y)

In [40]:
sk_X.shape

(506, 10)

SelectKBest에서 지정한 k개의 feature가 뽑힌다.

In [41]:
X.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [42]:
sk.get_feature_names_out()

array(['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'RAD', 'TAX', 'PTRATIO',
       'LSTAT'], dtype=object)

In [43]:
rf_cv = cross_val_score(rf, sk_X, y, scoring = 'neg_mean_squared_error', cv = KFold())

In [45]:
abs(rf_cv.mean())

21.959984864434084

***
#### 2) Classification

In [69]:
data = load_breast_cancer()

In [70]:
X = pd.DataFrame(data.data, columns = data.feature_names)
y = data.target

In [71]:
X.shape

(569, 30)

In [72]:
rf = RandomForestClassifier(random_state = 42)

In [73]:
sk = SelectKBest(chi2, k = 10)
sk_X = sk.fit_transform(X, y)

In [75]:
rf_cv = cross_val_score(rf, sk_X, y, scoring = 'roc_auc', cv = StratifiedKFold())

In [76]:
rf_cv.mean()

0.9850525959373814

***
# SelectFromModel
***
- feature importance에서 지정한 임계치 보다 큰 feature들을 사용한다.
- 트리 기반의 모델은 L1 규제(Lasso)를 일반적으로 사용한다.
- 임계치는 중위값, 평균 등의 값으로 설정이 가능하다.
- 단변량인 SelectKBest와 달리 모든 특성을 고려하므로 상호작용 부분을 반영 할 수 있다.
***

#### 1) Regression

In [84]:
data = load_boston()

In [85]:
X = pd.DataFrame(data.data, columns = data.feature_names)
y = data.target

In [86]:
X.shape

(506, 13)

In [87]:
rf = RandomForestRegressor(random_state = 42)

In [88]:
sfm = SelectFromModel(rf, threshold = 'median')

In [89]:
sfm.fit(X, y)

SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                threshold='median')

In [90]:
sfm_cols = sfm.get_feature_names_out()

In [91]:
rf_cv = cross_val_score(rf, X[sfm_cols.tolist()], y, scoring = 'neg_mean_squared_error', cv = KFold())

In [92]:
abs(rf_cv.mean())

20.981635685497956

***
#### 2) Classification

In [53]:
data = load_breast_cancer()

In [54]:
X = pd.DataFrame(data.data, columns = data.feature_names)
y = data.target

In [55]:
X.shape

(569, 30)

In [56]:
rf = RandomForestClassifier(random_state = 42)

In [57]:
sfm = SelectFromModel(rf, threshold = 'median')

In [58]:
sfm.fit(X, y)

SelectFromModel(estimator=RandomForestClassifier(random_state=42),
                threshold='median')

In [60]:
sfm_cols = sfm.get_feature_names_out()

In [67]:
rf_cv = cross_val_score(rf, X[sfm_cols.tolist()], y, scoring = 'roc_auc', cv = StratifiedKFold())

In [68]:
rf_cv.mean()

0.9907208106520258